In [1]:
import os
import pandas as pd
import yaml

In [2]:
with open("../../enriched_experiments_3.yml", "r") as f:
    exps = yaml.safe_load(f)

In [3]:
experiments = exps["experiments"]

In [4]:
experiments[-1].keys()

dict_keys(['slurm_id', 'experiment_nm', 'test_nm', 'wandb_urls', 'test/vaec_test5_vit_2/loss', 'test/vaec_test1_vit_2/loss', 'test/vaec_test2_vit_2/loss', 'val/vaec_vit/loss', 'test/vaec_test4_vit_2/loss', 'val/loss', 'test/vaec_test3_vit_2/loss', 'train/vaec_vit/loss_epoch', 'test/vaec_test5_vit_2/accuracy_vaec', 'test/vaec_test2_vit_2/accuracy_vaec', 'val/vaec_vit/accuracy_vaec', 'test/vaec_test1_vit_2/accuracy_vaec', 'test/vaec_test3_vit_2/accuracy_vaec', 'train/vaec_vit/accuracy_vaec_epoch', 'test/vaec_test4_vit_2/accuracy_vaec', 'max_epoch', 'loss/best_ckpt', 'datasets'])

In [5]:
experiments

[{'slurm_id': [914969],
  'experiment_nm': 'bongard_hoi_combined_v4_agg',
  'test_nm': 'baseline',
  'wandb_urls': ['https://wandb.ai/avr_universal/AVR_universal/runs/oqxw26p1'],
  'val/bongard_hoi_vit/loss': {'best_loss': 0.47659042477607727,
   'best_loss_epoch': 1,
   'best_loss_step': 2881,
   'best_slurm_id': 914969},
  'test/bongard_hoi_unseen-seen_vit_2/loss': {'best_loss': 0.5548768043518066,
   'best_loss_epoch': 0,
   'best_loss_step': 0,
   'best_slurm_id': 914969},
  'test/bongard_hoi_unseen-unseen_vit/loss': {'best_loss': 0.5013670921325684,
   'best_loss_epoch': 0,
   'best_loss_step': 0,
   'best_slurm_id': 914969},
  'val/loss': {'best_loss': 0.47666460275650024,
   'best_loss_epoch': 1,
   'best_loss_step': 2881,
   'best_slurm_id': 914969},
  'test/bongard_hoi_seen-seen_vit_2/loss': {'best_loss': 0.5340116620063782,
   'best_loss_epoch': 0,
   'best_loss_step': 0,
   'best_slurm_id': 914969},
  'test/bongard_hoi_seen-unseen_vit_2/loss': {'best_loss': 0.468476295471191

In [6]:
list(filter(lambda x: "accuracy" in x, experiments[-1].keys()))

['test/vaec_test5_vit_2/accuracy_vaec',
 'test/vaec_test2_vit_2/accuracy_vaec',
 'val/vaec_vit/accuracy_vaec',
 'test/vaec_test1_vit_2/accuracy_vaec',
 'test/vaec_test3_vit_2/accuracy_vaec',
 'train/vaec_vit/accuracy_vaec_epoch',
 'test/vaec_test4_vit_2/accuracy_vaec']

In [7]:
# Get all runs that produced some accuracy results
unprocessed_count = 0
processed_count = 0
structured_results = []
for experiment in experiments:
    accuracy_cols = [key for key in experiment.keys() if "accuracy" in key]
    if len(accuracy_cols) == 0:
        # TODO: later collect STSN runs
        unprocessed_count += 1
        continue
    structured_results.append(
        {col: experiment[col]["best_loss"] for col in accuracy_cols}
        | {
            "loss/best_ckpt": experiment.get("loss/best_ckpt"),
            "wandb_urls": ",".join(experiment.get("wandb_urls")),
            "experiment_nm": experiment.get("experiment_nm"),
            "test_nm": experiment.get("test_nm"),
            "based_on/slurm_id": experiment.get("based_on", {}).get("slurm_id"),
            "max_epoch": experiment.get("max_epoch"),
        }
    )
    # TODO: enrich with information from "additional_inforamations" (TODO: add relevant fields in enrich_experiments.py)
    processed_count += 1
print(f"Processed {processed_count} experiments")
print(f"Unprocessed {unprocessed_count} experiments")

Processed 102 experiments
Unprocessed 0 experiments


In [8]:
# TODO: split by dataset (not mutually exclusive -- lets duplicate the information)
# VAEC/VASR/HOI/LOGO/LOGO+HOI/VAEC+VASR/LOGO+VAEC/HOI+VASR

# might be nice to add identifiers different than slurm_ids
# (autoincrement value would work but could be hard to
# automatically assign it so that it makes sense in paper)

In [9]:
df = pd.DataFrame(structured_results)

In [10]:
df

,test/bongard_hoi_seen-seen_vit_2/accuracy_bongard_hoi,val/bongard_hoi_vit/accuracy_bongard_hoi,train/bongard_hoi_vit/accuracy_bongard_hoi_epoch,test/bongard_hoi_unseen-seen_vit_2/accuracy_bongard_hoi,test/bongard_hoi_seen-unseen_vit_2/accuracy_bongard_hoi,test/bongard_hoi_unseen-unseen_vit/accuracy_bongard_hoi,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,...,test/bongard_logo_test_ff_vit_2/accuracy_vasr,test/bongard_logo_test_hd_novel_vit_2/accuracy_vaec,test/bongard_logo_test_hd_comb_vit_2/accuracy_vaec,test/bongard_logo_test_ff_vit_2/accuracy_vaec,test/bongard_logo_test_bd_vit_2/accuracy_vaec,test/vaec_test1_vit_2/accuracy_vasr,test/vaec_test2_vit_2/accuracy_vasr,test/vaec_test4_vit_2/accuracy_vasr,test/vaec_test3_vit_2/accuracy_vasr,test/vaec_test5_vit_2/accuracy_vasr
0,0.774799,0.832452,0.979124,0.751644,0.842601,0.805211,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,baseline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,baseline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,baseline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,baseline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,svrt_combined_v4,baseline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,hoi_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,hoi_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,0.704200,0.800705,0.784167,0.702104,0.822850,0.751861,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,vasr_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,vasr_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# normalize columns

In [12]:
cols = df.columns

In [13]:
cols

Index(['test/bongard_hoi_seen-seen_vit_2/accuracy_bongard_hoi',
       'val/bongard_hoi_vit/accuracy_bongard_hoi',
       'train/bongard_hoi_vit/accuracy_bongard_hoi_epoch',
       'test/bongard_hoi_unseen-seen_vit_2/accuracy_bongard_hoi',
       'test/bongard_hoi_seen-unseen_vit_2/accuracy_bongard_hoi',
       'test/bongard_hoi_unseen-unseen_vit/accuracy_bongard_hoi',
       'loss/best_ckpt', 'wandb_urls', 'experiment_nm', 'test_nm',
       'based_on/slurm_id', 'max_epoch',
       'test/bongard_logo_test_bd_vit_2/accuracy_bongard_logo',
       'test/bongard_logo_test_hd_novel_vit_2/accuracy_bongard_logo',
       'test/bongard_logo_test_ff_vit_2/accuracy_bongard_logo',
       'test/bongard_logo_test_hd_comb_vit_2/accuracy_bongard_logo',
       'val/bongard_logo_vit/accuracy_bongard_logo',
       'train/bongard_logo_vit/accuracy_bongard_logo_epoch',
       'val/vasr_vit/accuracy_vasr', 'train/vasr_vit/accuracy_vasr_epoch',
       'test/vasr_vit/accuracy_vasr', 'test/vaec_test5_vit_2/acc

In [14]:
train_acc = [col for col in cols if "train" in col and "accuracy" in col]
val_acc = [col for col in cols if "val" in col and "accuracy" in col]

In [15]:
datasets = ["bongard_logo", "bongard_hoi", "vaec", "vasr", "iraven", "svrt"]

In [16]:
for dataset in datasets:
    _cols = [col for col in train_acc if dataset in col]
    df[f"train/{dataset}/accuracy"] = df[_cols].bfill(axis=1).iloc[:, 0]

    _cols = [col for col in val_acc if dataset in col]
    df[f"val/{dataset}/accuracy"] = df[_cols].bfill(axis=1).iloc[:, 0]

In [17]:
df = df.drop(
    columns=set(train_acc) - {f"train/{dataset}/accuracy" for dataset in datasets}
)
df = df.drop(columns=set(val_acc) - {f"val/{dataset}/accuracy" for dataset in datasets})

In [18]:
test_acc = [col for col in cols if "test" in col and "accuracy" in col]

In [19]:
test_types = [
    ("bongard_logo", "test_bd"),
    ("bongard_logo", "test_ff"),
    ("bongard_logo", "test_hd_comb"),
    ("bongard_logo", "test_hd_novel"),
    ("vaec", "test1"),
    ("vaec", "test2"),
    ("vaec", "test3"),
    ("vaec", "test4"),
    ("vaec", "test5"),
    ("bongard_hoi", "seen-seen"),
    ("bongard_hoi", "seen-unseen"),
    ("bongard_hoi", "unseen-seen"),
    ("bongard_hoi", "unseen-unseen"),
    ("vasr", "vasr"),
    ("iraven", "iraven"),
    ("svrt", "svrt")
]

In [20]:
for dataset, test_type in test_types:
    _cols = [col for col in test_acc if test_type in col]
    df[f"test/{dataset}/{test_type}"] = df[_cols].bfill(axis=1).iloc[:, 0]

In [21]:
df = df.drop(columns=test_acc)

In [22]:
df.columns

Index(['loss/best_ckpt', 'wandb_urls', 'experiment_nm', 'test_nm',
       'based_on/slurm_id', 'max_epoch', 'train/bongard_logo/accuracy',
       'val/bongard_logo/accuracy', 'train/bongard_hoi/accuracy',
       'val/bongard_hoi/accuracy', 'train/vaec/accuracy', 'val/vaec/accuracy',
       'train/vasr/accuracy', 'val/vasr/accuracy', 'train/iraven/accuracy',
       'val/iraven/accuracy', 'train/svrt/accuracy', 'val/svrt/accuracy',
       'test/bongard_logo/test_bd', 'test/bongard_logo/test_ff',
       'test/bongard_logo/test_hd_comb', 'test/bongard_logo/test_hd_novel',
       'test/vaec/test1', 'test/vaec/test2', 'test/vaec/test3',
       'test/vaec/test4', 'test/vaec/test5', 'test/bongard_hoi/seen-seen',
       'test/bongard_hoi/seen-unseen', 'test/bongard_hoi/unseen-seen',
       'test/bongard_hoi/unseen-unseen', 'test/vasr/vasr',
       'test/iraven/iraven', 'test/svrt/svrt'],
      dtype='object')

In [23]:
df.head()

,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_logo/accuracy,val/bongard_logo/accuracy,train/bongard_hoi/accuracy,val/bongard_hoi/accuracy,...,test/vaec/test3,test/vaec/test4,test/vaec/test5,test/bongard_hoi/seen-seen,test/bongard_hoi/seen-unseen,test/bongard_hoi/unseen-seen,test/bongard_hoi/unseen-unseen,test/vasr/vasr,test/iraven/iraven,test/svrt/svrt
0,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,baseline,None,101,NaN,NaN,0.979124,0.832452,...,NaN,NaN,NaN,0.774799,0.842601,0.751644,0.805211,NaN,NaN,NaN
1,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,baseline,None,231,0.916344,0.797778,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,baseline,None,81,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.563144,NaN,NaN
3,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,baseline,None,116,NaN,NaN,NaN,NaN,...,0.609562,0.564896,0.588477,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,svrt_combined_v4,baseline,None,124,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9575


In [24]:
for dataset, test_type in test_types:
    _dataset_cols = [col for col in df.columns if dataset in col and "test" not in col]
    cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]
    display(
        df[[f"test/{dataset}/{test_type}"] + cols]
        .sort_values(f"test/{dataset}/{test_type}", ascending=False)
        .head(5)
    )

,test/bongard_logo/test_bd,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_logo/accuracy,val/bongard_logo/accuracy
11,0.841667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,asymmetric_only_parameter_search,None,116,0.976774,0.778889
23,0.841667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4,diff_aggregation_parameter_search,None,197,0.951505,0.786667
53,0.839583,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,bongard_hoi_logo_fine_tune,None,367,0.923763,0.784444
41,0.839583,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,small_transformer_scoring,None,101,0.987097,0.763333
45,0.839583,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,None,102,0.979677,0.777778


,test/bongard_logo/test_ff,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_logo/accuracy,val/bongard_logo/accuracy
23,0.715000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4,diff_aggregation_parameter_search,None,197,0.951505,0.786667
53,0.706667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,bongard_hoi_logo_fine_tune,None,367,0.923763,0.784444
27,0.685000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,no_context_parameter_search,None,146,0.968817,0.784444
75,0.680000,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_bongard_logo_combined_v5,common_relationals_separate_scoring,None,50,0.972323,0.772222
45,0.678333,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,None,102,0.979677,0.777778


,test/bongard_logo/test_hd_comb,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_logo/accuracy,val/bongard_logo/accuracy
45,0.7250,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,None,102,0.979677,0.777778
27,0.7100,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,no_context_parameter_search,None,146,0.968817,0.784444
41,0.6925,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,small_transformer_scoring,None,101,0.987097,0.763333
11,0.6775,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,asymmetric_only_parameter_search,None,116,0.976774,0.778889
23,0.6750,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4,diff_aggregation_parameter_search,None,197,0.951505,0.786667


,test/bongard_logo/test_hd_novel,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_logo/accuracy,val/bongard_logo/accuracy
41,0.765625,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,small_transformer_scoring,None,101,0.987097,0.763333
45,0.765625,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,None,102,0.979677,0.777778
11,0.753125,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,asymmetric_only_parameter_search,None,116,0.976774,0.778889
1,0.743750,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,baseline,None,231,0.916344,0.797778
27,0.743750,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,no_context_parameter_search,None,146,0.968817,0.784444


,test/vaec/test1,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/vaec/accuracy,val/vaec/accuracy
67,0.672817,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v4,common_relationals_common_scoring,None,107,0.935132,0.672576
64,0.671335,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_bongard_logo_combined_v4,separate_relationals_common_scoring,None,166,0.946519,0.671489
70,0.669062,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_bongard_logo_combined_v4,common_relationals_common_scoring,None,106,0.951678,0.669087
76,0.662374,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_bongard_logo_combined_v5,common_relationals_separate_scoring,None,129,0.993705,0.662564
73,0.654906,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v5,common_relationals_separate_scoring,None,103,0.991683,0.661649


...

,test/bongard_hoi/unseen-seen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_hoi/accuracy,val/bongard_hoi/accuracy
44,0.761727,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_transformer_v4,transformer_scoring,None,54,0.997917,0.835097
33,0.760631,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128_64,None,101,0.987891,0.837155
32,0.754494,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128,None,101,0.977909,0.827454
0,0.751644,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,baseline,None,101,0.979124,0.832452
80,0.749452,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v4_agg,partial_freezing_hoi_0_1,None,157,0.964107,0.829512


,test/bongard_hoi/unseen-unseen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/bongard_hoi/accuracy,val/bongard_hoi/accuracy
6,0.818859,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,symmetric_only_parameter_search,None,119,0.920793,0.830394
31,0.815757,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_64,None,100,0.986589,0.820988
33,0.815136,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128_64,None,101,0.987891,0.837155
73,0.813275,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v5,common_relationals_separate_scoring,None,103,0.985982,0.831276
80,0.809553,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v4_agg,partial_freezing_hoi_0_1,None,157,0.964107,0.829512


,test/vasr/vasr,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/vasr/accuracy,val/vasr/accuracy
28,0.595039,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,no_context_parameter_search,None,100,0.956527,0.596076
2,0.563144,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,baseline,None,81,0.931823,0.561884
58,0.562642,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,vasr_vaec_fine_tune,None,61,0.913625,0.563879
37,0.561662,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,mlp_hidden_dim_256_128,None,75,0.977587,0.561408
89,0.553994,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_vaec_combined_v4,partial_freezing_vaec_0_1,None,64,0.911837,0.569186


,test/iraven/iraven,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/iraven/accuracy,val/iraven/accuracy
5,0.650822,https://wandb.ai/avr_universal/AVR_universal/r...,iraven_combined_v4,baseline,None,499,0.857779,0.650284
0,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,baseline,None,101,NaN,NaN
1,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,baseline,None,231,NaN,NaN
2,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,baseline,None,81,NaN,NaN
3,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,baseline,None,116,NaN,NaN


,test/svrt/svrt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/svrt/accuracy,val/svrt/accuracy
4,0.9575,https://wandb.ai/avr_universal/AVR_universal/r...,svrt_combined_v4,baseline,None,124,0.9802,0.9705
0,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,baseline,None,101,NaN,NaN
1,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,baseline,None,231,NaN,NaN
2,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,baseline,None,81,NaN,NaN
3,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,baseline,None,116,NaN,NaN


In [37]:
df.to_csv("../../structured_results.csv", index=False)

In [36]:
df[["test/bongard_logo/test_bd",	"test/bongard_logo/test_ff",	"test/bongard_logo/test_hd_comb",	"test/bongard_logo/test_hd_novel"]] = df[["test/bongard_logo/test_bd",	"test/bongard_logo/test_ff",	"test/bongard_logo/test_hd_comb",	"test/bongard_logo/test_hd_novel"]].apply(lambda x: round(100*x,1))

In [33]:
df[["test/vaec/test1", "test/vaec/test2", "test/vaec/test3", "test/vaec/test4", "test/vaec/test5", "test/bongard_hoi/seen-seen", "test/bongard_hoi/seen-unseen", "test/bongard_hoi/unseen-seen", "test/bongard_hoi/unseen-unseen"]] = df[["test/vaec/test1", "test/vaec/test2", "test/vaec/test3", "test/vaec/test4", "test/vaec/test5", "test/bongard_hoi/seen-seen", "test/bongard_hoi/seen-unseen", "test/bongard_hoi/unseen-seen", "test/bongard_hoi/unseen-unseen"]].apply(lambda x: round(x*100,1))


In [26]:
# split by dataset 

In [27]:
#df = round(df * 100,1)